### EfficientNetによる異常検知　kerasによる実装

In [122]:
#import efficientNet

import numpy as np
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis
import tensorflow as tf
from keras.models import load_model  
from keras.layers import GlobalAveragePooling2D
import efficientnet as efn

model = efn.EfficientNetB4(weights='imagenet') 

78905344/78899776 [==============================] - 58s 1us/step


In [132]:
#画像の大きさ(入力サイズ)を変えたら、どうなるの？
model.summary()

                
__________________________________________________________________________________________________
conv2d_235 (Conv2D)             (None, 1, 1, 68)     111044      lambda_59[0][0]                  
__________________________________________________________________________________________________
swish_178 (Swish)               (None, 1, 1, 68)     0           conv2d_235[0][0]                 
__________________________________________________________________________________________________
conv2d_236 (Conv2D)             (None, 1, 1, 1632)   112608      swish_178[0][0]                  
__________________________________________________________________________________________________
activation_61 (Activation)      (None, 1, 1, 1632)   0           conv2d_236[0][0]                 
__________________________________________________________________________________________________
multiply_59 (Multiply)          (None, 12, 12, 1632) 0           activation_61[0][0]        

In [124]:
for a in model.layers:
    print(a.output)

4, 24, 672), dtype=float32)
Tensor("swish_143/swish_f32:0", shape=(?, 24, 24, 672), dtype=float32)
Tensor("depthwise_conv2d_48/depthwise:0", shape=(?, 24, 24, 672), dtype=float32)
Tensor("batch_normalization_144/cond/Merge:0", shape=(?, 24, 24, 672), dtype=float32)
Tensor("swish_144/swish_f32:0", shape=(?, 24, 24, 672), dtype=float32)
Tensor("lambda_48/Mean:0", shape=(?, 1, 1, 672), dtype=float32)
Tensor("conv2d_191/BiasAdd:0", shape=(?, 1, 1, 28), dtype=float32)
Tensor("swish_145/swish_f32:0", shape=(?, 1, 1, 28), dtype=float32)
Tensor("conv2d_192/BiasAdd:0", shape=(?, 1, 1, 672), dtype=float32)
Tensor("activation_50/Sigmoid:0", shape=(?, 1, 1, 672), dtype=float32)
Tensor("multiply_48/mul:0", shape=(?, 24, 24, 672), dtype=float32)
Tensor("conv2d_193/convolution:0", shape=(?, 24, 24, 112), dtype=float32)
Tensor("batch_normalization_145/cond/Merge:0", shape=(?, 24, 24, 112), dtype=float32)
Tensor("drop_connect_30/cond/Merge:0", shape=(?, 24, 24, 112), dtype=float32)
Tensor("add_30/add:0

In [129]:
model.layers[0].output

<tf.Tensor 'input_3:0' shape=(?, 380, 380, 3) dtype=float32>

In [93]:
#keras 各層の出力を抽出する。
add_layer_list = [[num,a]for num,a in enumerate(model.layers) if "Add" in str(a)]
for num in range(len(add_layer_list)):
    print(add_layer_list[num][0],add_layer_list[num][1].output)



44 Tensor("add_10/add:0", shape=(?, 56, 56, 24), dtype=float32)
74 Tensor("add_11/add:0", shape=(?, 28, 28, 40), dtype=float32)
104 Tensor("add_12/add:0", shape=(?, 14, 14, 80), dtype=float32)
120 Tensor("add_13/add:0", shape=(?, 14, 14, 80), dtype=float32)
150 Tensor("add_14/add:0", shape=(?, 14, 14, 112), dtype=float32)
166 Tensor("add_15/add:0", shape=(?, 14, 14, 112), dtype=float32)
196 Tensor("add_16/add:0", shape=(?, 7, 7, 192), dtype=float32)
212 Tensor("add_17/add:0", shape=(?, 7, 7, 192), dtype=float32)
228 Tensor("add_18/add:0", shape=(?, 7, 7, 192), dtype=float32)


In [51]:
swish_layer_list = [a for a in model.layers if "(?, 112, 112, 16)" in str(a)]
for num in range(len(swish_layer_list)):
    print(swish_layer_list[num].output)


In [107]:
#efficientnetの出力を表示する。
from skimage import io
from skimage.transform import resize
img = io.imread("./test.jpg")
img_resize = resize(img,(224,224,3))
img_resize = np.expand_dims(img_resize,axis=0)

img_resize2 = np.vstack((img_resize,img_resize))


In [88]:
#お試し、何をやっているのかを確認する。
from keras import backend as K 
from keras.models import Model
inp = model.input# input placeholder
outputs = [layer.output for layer in model.layers]# all layer outputs
models = [Model(inputs=inp,outputs=out) for out in outputs[2:5]]

new_models = []
for num,tmp in enumerate(models):
    input = model.input
    x = input
    x = tmp(x)
    output = GlobalAveragePooling2D()(x)
    new_models.append(Model(input,output))

# Testing
#test = np.random.random(input_shape)[np.newaxis,...]
layer_outs = [ tmp_model.predict(img_resize) for tmp_model in new_models]
print(layer_outs[0].shape)
print(layer_outs[1].shape)
print(layer_outs[2].shape)


(1, 32)
(1, 32)
(1, 32)


In [109]:
#各層からの出力をリストにまとめて出力する。
from keras.models import Model

def extract_features(model,input_data):
    """
    各層の特徴量を算出する関数。
    model:efficientnet(b0~b7)
    input:img
    """

    feat_list = []

    select_layers = [6,14,44,74,120,166,228,-8,-5]#each layer nums
    
    #get each layer models
    new_models = []
    for num,layer_num in enumerate(select_layers):
        input = model.input
        x = input
        x = model.layers[layer_num].output
        output = GlobalAveragePooling2D()(x)
        new_models.append(Model(input,output))

    #get layer features
    for num, tmp in enumerate(new_models):
        feat_list.append(tmp.predict(input_data))

    
    return feat_list


lists = extract_features(model,img_resize2)



In [111]:
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from matplotlib.pyplot as plt

#学習画像の特徴量を保存するリスト
train_outputs = [[] for _ in range(9)]
test_outputs = [[] for _ in range(9)]

#何かしらのデータに対して適用する。
for data in train_datas:
    feats = extract_features(model,data)
    
    for f_idx, feat in enumerate(feats):
        train_outputs[f_idx].append(feat)
    
    # fitting a multivariate gaussian to features extracted from every level of ImageNet pre-trained model
    for t_idx, train_output in enumerate(train_outputs):
        mean = np.mean(np.array(train_output),axis=0)#channel数分の平均値を算出する。

        #channel数分の分散、共分散行列を求める。
        #例えば、第１層の出力では、32次元の各軸の平均と分散を求めることになる。
        cov = LedoitWolf().fit(np.array(train_output)).covariance_
        train_outputs[t_idx] = [mean, cov]

########################################################
#テストデータでも同様に特徴量を計算する。
#テストデータでは画像情報とスコアを一致させるために、正常か異常かを保存しておくこと
gt_list = []

for data in test_datas:
    gt_list.extend(y.cpu().detach().numpy())#ラベルつけを保存しておく。
    feats = extract_features(model,data)
    
    for f_idx, feat in enumerate(feats):
        test_outputs[f_idx].append(feat)
    
    # fitting a multivariate gaussian to features extracted from every level of ImageNet pre-trained model
    for t_idx, test_output in enumerate(test_outputs):
        test_outputs[t_idx] = np.array(test_output)

########################################################
#マハラノビス距離の計算
dist_list = []
for t_idx, test_output in enumerate(test_outputs):
    mean = train_outputs[t_idx][0]#学習データの平均
    cov_inv = np.linalg.inv(train_outputs[t_idx][1])#学習データの分散
    dist = [mahalanobis(sample, mean, cov_inv) for sample in test_output]
    dist_list.append(np.array(dist))

# Anomaly score is followed by unweighted summation of the Mahalanobis distances
#各画像に対する異常度（距離）が出てくる。100枚の画像だったら、100個の特徴量
scores = np.sum(np.array(dist_list), axis=0)

# calculate image-level ROC AUC score
fpr, tpr, _ = roc_curve(gt_list, scores)
roc_auc = roc_auc_score(gt_list, scores)
total_roc_auc.append(roc_auc)
print('%s ROCAUC: %.3f' % (class_name, roc_auc))
plt.plot(fpr, tpr, label='%s ROCAUC: %.3f' % (class_name, roc_auc))


array([[ 0.21044457, -0.25377545,  1.8010106 ,  1.5387206 ,  3.7707725 ,
         2.1740406 ,  2.300516  ,  0.525279  ,  1.6948812 ,  1.731594  ,
         3.0381918 ,  1.9850122 ,  5.3983293 ,  1.4301822 ,  2.4775245 ,
         2.7891462 ,  7.4035773 ,  0.7185514 ,  1.6730255 ,  0.47469485,
         6.6796446 ,  1.9404509 ,  0.28501004,  4.4467235 ,  1.5025014 ,
        -0.24201126,  1.6556723 ,  2.069965  ,  0.2935092 ,  0.32747853,
         1.583767  ,  1.3538224 ],
       [ 0.21044457, -0.25377545,  1.8010106 ,  1.5387206 ,  3.7707725 ,
         2.1740406 ,  2.300516  ,  0.525279  ,  1.6948812 ,  1.731594  ,
         3.0381918 ,  1.9850122 ,  5.3983293 ,  1.4301822 ,  2.4775245 ,
         2.7891462 ,  7.4035773 ,  0.7185514 ,  1.6730255 ,  0.47469485,
         6.6796446 ,  1.9404509 ,  0.28501004,  4.4467235 ,  1.5025014 ,
        -0.24201126,  1.6556723 ,  2.069965  ,  0.2935092 ,  0.32747853,
         1.583767  ,  1.3538224 ]], dtype=float32)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.